In [6]:
import onnx
import pandas as pd
import numpy as np
import onnxruntime as rt
import torch

/home/manoj/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
i = 10
i += 5


df = pd.read_csv("/home/manoj/sdn/onos-apps/network_data/models/v1/d_star_10000_1_5_u.csv")
x = []
for j in range(5):
    x.append(df[df['id'] == i-j].sort_values(by="Sno").to_numpy()[:,2:] / np.array([10000 , 10000 , 10000000 ]) )

x = np.array(x)
x = np.swapaxes(x,0,1)

y = df[df['id'] == i].sort_values(by="Sno").to_numpy()[:,2:] / np.array([10000 , 10000 , 10000000])
x.shape , y.shape

((10, 5, 3), (10, 3))

In [11]:
l = 0
for i in range(5):
    print( x[l,i,:])
    
print(y[l] * [10000 , 10000 , 10000000] )

[4.348e-02 1.460e-02 2.580e-05]
[1.701e-02 8.012e-03 3.205e-05]
[3.090e-02 1.062e-02 2.575e-05]
[2.842e-02 1.012e-02 2.930e-05]
[1.551e-02 7.912e-03 3.560e-05]
[434.8 146.  258. ]


In [12]:
md = rt.InferenceSession("/home/manoj/sdn/onos-apps/network_data/models/v1/gru_model_v22.onnx", providers=rt.get_available_providers())

print("input name='{}' and shape={}".format(md.get_inputs()[0].name, md.get_inputs()[0].shape))
print("output name='{}' and shape={}".format(md.get_outputs()[0].name, md.get_outputs()[0].shape))

input_name = md.get_inputs()[0].name
label_name = md.get_outputs()[0].name

ot = md.run([label_name], {input_name: x.astype(np.float32) })
ot

input name='input' and shape=['batch_size', 5, 3]
output name='output' and shape=['batch_size', 3]


[array([[ 1.41864523e-01,  9.82625782e-02, -1.98180415e-02],
        [ 9.50355530e-02,  8.52227211e-04,  1.08417058e+00],
        [ 1.42855465e-01,  1.03292719e-01, -2.03820020e-02],
        [ 1.57542244e-01,  1.05555475e-01, -1.97724216e-02],
        [ 1.56160355e-01,  9.13751870e-02, -2.06847154e-02],
        [ 1.36558756e-01,  9.89518464e-02, -2.13287733e-02],
        [-4.76214886e-02, -1.03371799e-01,  8.10677052e-01],
        [ 1.24264866e-01,  7.88527280e-02, -2.22922992e-02],
        [ 1.40589118e-01,  9.26078558e-02, -2.00777948e-02],
        [ 1.38362527e-01,  9.27267894e-02, -2.16131695e-02]], dtype=float32)]

In [13]:
ot[0][l] * [10000 , 10000 , 10000000]

array([   1418.64523292,     982.62578249, -198180.41473627])

In [14]:
((ot[0][l] - y[l])**2).sum()**0.5

0.1306626169453595